The following cell performs sweeps of the lambda value. Within the `validation_train` loop, the following steps are executed for each value in the `alpha_grid`:

- The function `lasso_regression` is called to fit a lasso regression model using the subject's transformed SC and FC matrices and the current `alpha` value.
- The resulting transformation rules are saved to disk in a directory specific to the current window size and subject.
- GPU memory is cleared and Python garbage collection is triggered to manage resources efficiently.

This process is repeated for each regularization parameter in the grid, enabling systematic exploration of model performance across different penalty strengths.

In [ ]:
## SC Density Sweeps - Helper Functions ##
import cupy as cp
from transformations import symmetric_modification


def calc_fc(ts_data, time_steps, starting_time):
    """
    Calculate the functional connectivity matrix from time series data.
    """
    n = ts_data.shape[1]
    fc_matrix = cp.zeros((n, n))
    fc_matrix = cp.corrcoef(ts_data[starting_time:time_steps + starting_time, :], rowvar=False)
    return fc_matrix.astype(cp.float32)

def load_connectomes(subject_id, time_steps=-1, starting_time=0):
    """
    Load the data for a given subject and window ratio.
    """
    # Data paths
    function_ts_path = f"ts_data/iPA_183/ts/ts_sub-{subject_id}_183.txt"
    sc_path = f"ts_data/iPA_183/sc/sub-{subject_id}_SC.csv"

    # Get sc & fc matrices and return them
    ts_data = cp.loadtxt(function_ts_path)
    fc = calc_fc(ts_data, time_steps, starting_time)
    sc = cp.loadtxt(sc_path)
    cp.fill_diagonal(sc, cp.mean(sc))
    return sc.astype(cp.float32), fc


## SC Density Sweeps - Train ##
import gc
from lin_gen_model import calc_alpha_grid, lasso_regression, Subject
from transformations import symmetric_modification
import inout


def validation_train(subject_id):
    """
    Train the subject-level model for a given subject ID at different window sizes and penalty values.
    """
    time_steps = 587 
    sc, fc = load_connectomes(subject_id, time_steps=time_steps)
    subject = Subject(subject_id, sc, fc, symmetric_modification)
    alpha_grid = calc_alpha_grid(subject.transformed_sc, subject.transformed_fc, 20)

    print(f"Window = {time_steps}, alpha_grid = {alpha_grid}")

    for num, alpha in enumerate(alpha_grid):
        rules = lasso_regression(subject.transformed_sc, subject.transformed_fc, alpha, max_iter=100)
        outdir = f"sc_density_sweeps/{time_steps}/"
        inout.check_paths(outdir)
        cp.savetxt(f"{outdir}rules_sub-{subject_id}_lambda-{num}", rules)

        cp.get_default_memory_pool().free_all_blocks()
        cp.get_default_pinned_memory_pool().free_all_blocks()
        # Force Python garbage collection:
        gc.collect()

SUBJECT_IDs = ['032301', '032304', '032307']
for subject_id in SUBJECT_IDs:
    validation_train(subject_id)

This cell implements the training workflow for subject-level models across multiple subjects using structural connectivity (SC) density sweeps. For each subject ID in the `SUBJECT_IDs` list, it loads the subject's SC and functional connectivity (FC) matrices, applies a symmetric transformation, and computes a grid of regularization parameters (`alpha_grid`). For each value in this grid, it fits a lasso regression model to learn transformation rules, saves the resulting rules to disk, and manages GPU memory and garbage collection to optimize resource usage. This process enables systematic exploration of model performance across different regularization strengths and window sizes.

In [ ]:
## SC Density Sweeps - Plot ##
import cupy as cp
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from transformations import inverse_symmetric_modification

def make_validation_plots(subject_id):
    fig1, axs_both_windows = plt.subplots(2, 4, figsize=(20, 10))
    axs_both_windows = axs_both_windows.flatten()

    fig2, axs_same_window = plt.subplots(2, 4, figsize=(20, 10))
    axs_same_window = axs_same_window.flatten()

    fig3, axs_diff_window = plt.subplots(2, 4, figsize=(20, 10))
    axs_diff_window = axs_diff_window.flatten()

    for plot_indx, window_size in enumerate((16, 65, 98, 130, 196, 326, 587)):
        # Data paths
        function_ts_path = f"ts_data/iPA_183/ts/ts_sub-{subject_id}_183.txt"
        ts_data = cp.loadtxt(function_ts_path)

        fc_same_window = calc_fc(ts_data, window_size, 0)
        fc_diff_window = calc_fc(ts_data, window_size, 652-window_size)

        sc_path = f"ts_data/iPA_183/sc/sub-{subject_id}_SC.csv"
        sc = cp.loadtxt(sc_path)
        cp.fill_diagonal(sc, 0)

        r2s_same_window = []
        r2s_diff_window = []
        sc_densities = []
        for indx in range(20):
            rules_path = f"sc_density_sweeps/{window_size}/rules_sub-{subject_id}_lambda-{indx}"
            rules = cp.loadtxt(rules_path)
            rules = inverse_symmetric_modification(rules, 183)

            predfc = sc @ rules @ sc

            r2_same_window = r2_score(fc_same_window.get().flatten(), predfc.get().flatten())
            r2s_same_window.append(r2_same_window)

            r2_diff_window = r2_score(fc_diff_window.get().flatten(), predfc.get().flatten())
            r2s_diff_window.append(r2_diff_window)

            density = cp.count_nonzero(rules) / (rules.shape[0] ** 2)
            sc_densities.append(density.get())

        axs_both_windows[plot_indx].scatter(sc_densities, r2s_same_window, color="blue")
        axs_both_windows[plot_indx].scatter(sc_densities, r2s_diff_window, color="red")
        axs_both_windows[plot_indx].set_title(f"R2 - Window Size: {window_size}/ 652")
        axs_both_windows[plot_indx].set_xlabel("Rules Density")

        axs_same_window[plot_indx].scatter(sc_densities, r2s_same_window, color="blue")
        axs_same_window[plot_indx].set_title(f"R2 - Window Size: {window_size}/ 652")
        axs_same_window[plot_indx].set_xlabel("Rules Density")

        axs_diff_window[plot_indx].scatter(sc_densities, r2s_diff_window, color="red")
        axs_diff_window[plot_indx].set_title(f"R2 - Window Size: {window_size}/ 652")
        axs_diff_window[plot_indx].set_xlabel("Rules Density")

    fig1.savefig(f"r2_density_both_windows_sub-{subject_id}.png")
    fig2.savefig(f"r2_density_same_window_sub-{subject_id}.png")
    fig3.savefig(f"r2_density_diff_window_sub-{subject_id}.png")

    plt.close(fig1)
    plt.close(fig2)
    plt.close(fig3)

SUBJECT_IDs = ['032304', '032302', '032307']
for subject_id in SUBJECT_IDs:
    make_validation_plots(subject_id)

This cell implements subject-level training using the Schaefer 100 parcellation. For each subject ID from 1 to 50, it loads the subject's structural connectivity (SC) and functional connectivity (FC) matrices, applies a symmetric transformation, and fits a model using a binary search to optimize the regularization parameter. The resulting transformation rules are saved for each subject. GPU memory is managed and garbage collection is performed after each subject to ensure efficient resource usage.

In [ ]:
## Subject-Level - Train ##
import gc
import cupy as cp
import inout
from inout import get_schaefer100_sc, get_schaefer100_fc
from lin_gen_model import Subject, binary_search_train
from transformations import symmetric_modification


def train_subject(subject_id):
    """
    Train the subject-level model for a given subject ID.
    """
    # Load the data
    sc, fc = get_schaefer100_sc(subject_id), get_schaefer100_fc(subject_id)
    subject = Subject(subject_id, sc, fc, symmetric_modification)
    rules, alpha = binary_search_train(subject.transformed_sc, subject.transformed_fc, max_iter=10)

    # Save the model
    sl_dir = "subject_level_log10"
    inout.check_paths(sl_dir)
    cp.savetxt(f"{sl_dir}/rules_sub-{subject_id}", rules)

    print(f"Subject {subject_id} finished training with alpha: {alpha}")
    print()

for subject_id in range(1, 51):
    train_subject(subject_id)

    cp.get_default_memory_pool().free_all_blocks()
    cp.get_default_pinned_memory_pool().free_all_blocks()
    # Force Python garbage collection:
    gc.collect()

This cell implements subject-level training using the Schaefer 100 parcellation, but applies a log base 10 transformation to the structural connectivity (SC) matrices before model fitting. For each subject ID from 1 to 50, it loads the subject's SC and functional connectivity (FC) matrices, transforms the SC matrix with `log10(sc + 1)`, applies a symmetric transformation, and fits a model using a binary search to optimize the regularization parameter. The resulting transformation rules are saved for each subject. GPU memory is managed and garbage collection is performed after each subject to ensure efficient resource usage.

In [ ]:
## Subject-Level - Train (log base 10) ##
import gc
import cupy as cp
import inout
from inout import get_schaefer100_sc, get_schaefer100_fc
from lin_gen_model import Subject, binary_search_train
from transformations import symmetric_modification


def train_subject(subject_id):
    """
    Train the subject-level model for a given subject ID.
    """
    # Load the data
    sc, fc = get_schaefer100_sc(subject_id), get_schaefer100_fc(subject_id)
    sc = cp.log10(sc + 1)
    subject = Subject(subject_id, sc, fc, symmetric_modification)
    rules, alpha = binary_search_train(subject.transformed_sc, subject.transformed_fc, max_iter=10)

    # Save the model
    sl_dir = "subject_level_log10"
    inout.check_paths(sl_dir)
    cp.savetxt(f"{sl_dir}/rules_sub-{subject_id}", rules)

    print(f"Subject {subject_id} finished training with alpha: {alpha}")
    print()

for subject_id in range(1, 51):
    train_subject(subject_id)

    cp.get_default_memory_pool().free_all_blocks()
    cp.get_default_pinned_memory_pool().free_all_blocks()
    # Force Python garbage collection:
    gc.collect()

This cell implements subject-level training where each subject's functional connectivity (FC) matrix is paired with the structural connectivity (SC) matrix from a different subject. For each subject ID, the FC is loaded for that subject, while the SC is loaded from another subject according to a deterministic mapping. The model is trained using these mismatched SC-FC pairs, and the resulting transformation rules are saved. GPU memory is managed and garbage collection is performed after each subject to ensure efficient resource usage. This approach allows for the assessment of how well models generalize when trained on SC from other individuals.

In [ ]:
## Subject-Level - Train w/ Other Subject's SC ##
import gc
import cupy as cp
import inout
from inout import get_schaefer100_sc, get_schaefer100_fc
from lin_gen_model import Subject, binary_search_train
from transformations import symmetric_modification


def train_subject(subject_id):
    """
    Train the subject-level model for a given subject ID.
    """
    fc = get_schaefer100_fc(subject_id)

    # Map each subject_id to another in a one-to-one function
    if subject_id <= 40:
        subject_id_sc = subject_id + 10
    else:
        subject_id_sc = (subject_id - 1) % 40 + 1
    
    # Load the data
    sc = get_schaefer100_sc(subject_id_sc)
    subject = Subject(subject_id, sc, fc, symmetric_modification)
    rules, alpha = binary_search_train(subject.transformed_sc, subject.transformed_fc, max_iter=10)

    # Save the model
    sl_dir = "subject_level_other_sc"
    inout.check_paths(sl_dir)
    cp.savetxt(f"{sl_dir}/rules_sc-{subject_id_sc}_fc-{subject_id}", rules)

    print(f"Subject {subject_id} FC w/ Subject {subject_id_sc} SC finished training with alpha: {alpha}")
    print()

for subject_id in range(1, 51):
    train_subject(subject_id)

    cp.get_default_memory_pool().free_all_blocks()
    cp.get_default_pinned_memory_pool().free_all_blocks()
    # Force Python garbage collection:
    gc.collect()

This cell implements subject-level training using null structural connectivity (SC) matrices. For each subject ID from 1 to 50, it loads the subject's functional connectivity (FC) matrix and iterates over 100 null SC matrices generated for that subject. Each null SC matrix is processed (with the medial wall removed), and a model is trained using the FC and the transformed null SC. The resulting transformation rules are saved for each null, allowing for statistical comparison against models trained with real SC. GPU memory is managed and garbage collection is performed after each null model to ensure efficient resource usage.

In [ ]:
## Subject-Level - Train w/ SC nulls ##
import gc
import cupy as cp
import inout
from inout import get_schaefer100_fc
from lin_gen_model import Subject, binary_search_train
from transformations import symmetric_modification

def train_subject(subject_id):
    """
    Train the subject-level model for a given subject ID.
    """
    fc = get_schaefer100_fc(subject_id)

    for null_num in range(0, 100):
    
        # Load the data
        sc = cp.loadtxt(f"sc_nulls/{subject_id}/null_X{null_num}")
        sc = inout.remove_medial_wall(sc)
        subject = Subject(subject_id, sc, fc, symmetric_modification)
        rules, alpha = binary_search_train(subject.transformed_sc, subject.transformed_fc, max_iter=10)

        # Save the model
        sl_dir = f"rule_nulls/{subject_id}"
        inout.check_paths(sl_dir)
        cp.savetxt(f"{sl_dir}/null_rules{null_num}", rules)

        print(f"Subject {subject_id}, null_num {null_num} finished training with alpha: {alpha}")
        print()
        gc.collect()


for subject_id in range(1, 51):
    train_subject(subject_id)

    cp.get_default_memory_pool().free_all_blocks()
    cp.get_default_pinned_memory_pool().free_all_blocks()
    # Force Python garbage collection:
    gc.collect()


This cell identifies statistically significant  rules at the subject level by comparing each subject's learned rules to a null distribution generated from 100 null models. For each subject, it loads the learned rules and the corresponding null distributions, computes p-values for each rule using a t-test, and sets non-significant rules (p > 0.05) to NaN. The resulting p-values and significant rules matrices are saved for further analysis.

In [ ]:
## Subject-Level - Find Significant Rules ##
import numpy as np
import os
from scipy.stats import t


def compute_pvals(distribution, matrix):
    n = matrix.shape[0]
    df = n - 1

    mean = distribution.mean(axis=1)
    std = distribution.std(axis=1, ddof=1)
    t_stat = (mean - matrix) / ((std + 1e-64) / np.sqrt(n))
    p_values = 2 * t.sf(np.abs(t_stat), df)
    return p_values

def compute_sig_rules(p_values, matrix):
    matrix[p_values > 0.05] = np.nan
    return matrix

for subject_id in range(1, 51):
    rules = np.loadtxt(f"subject_level/rules_sub-{subject_id}")

    distribution = []
    for null_num in range(100):
        if not os.path.exists(f"rule_nulls/{subject_id}/null_rules{null_num}"):
            continue
        null_rules = np.loadtxt(f"rule_nulls/{subject_id}/null_rules{null_num}")
        distribution.append(null_rules)
    distribution = np.vstack(distribution).T

    pvals = compute_pvals(distribution, rules)
    sig_rules = compute_sig_rules(pvals, rules)

    np.savetxt(f"p_values_sub-{subject_id}", pvals)
    np.savetxt(f"sig_rules_sub-{subject_id}", sig_rules)

This cell implements subject-level training using search information matrices derived from structural connectivity (SC). For each subject ID from 1 to 50, it loads the subject's functional connectivity (FC) matrix and the corresponding search information matrix (used as SC). The model is trained using these matrices and a symmetric transformation, and the resulting transformation rules are saved for each subject. GPU memory is managed and garbage collection is performed after each subject to ensure efficient resource usage. This approach allows for the assessment of how well search information can predict functional connectivity at the subject level.

In [ ]:
## Subject-Level - Train w/ Search Information ##
import gc
import cupy as cp
import inout
from inout import get_schaefer100_fc
from lin_gen_model import Subject, binary_search_train
from transformations import symmetric_modification

def train_subject(subject_id):
    """
    Train the subject-level model for a given subject ID.
    """
    fc = get_schaefer100_fc(subject_id)

    # Load the data
    sc = cp.loadtxt(f"search_information_from_sc/search_information_{subject_id}.txt")
    subject = Subject(subject_id, sc, fc, symmetric_modification)
    rules, alpha = binary_search_train(subject.transformed_sc, subject.transformed_fc, max_iter=10)

    # Save the model
    sl_dir = "subject_level_search_information/"
    inout.check_paths(sl_dir)
    cp.savetxt(f"{sl_dir}/rules_sub-{subject_id}", rules)

    print(f"Subject {subject_id} finished training with alpha: {alpha}")
    print()


for subject_id in range(1, 51):
    train_subject(subject_id)

    cp.get_default_memory_pool().free_all_blocks()
    cp.get_default_pinned_memory_pool().free_all_blocks()
    # Force Python garbage collection:
    gc.collect()


This cell implements group-level training using the Schaefer 100 parcellation. For each subject ID from 1 to 50, it loads the subject's structural connectivity (SC) and functional connectivity (FC) matrices, applies a log base 10 transformation to the SC matrix, and then applies a symmetric transformation. The transformed SC and FC matrices are collected across subjects and stacked to form group-level datasets. The code prepares these datasets for further group-level model fitting, such as regression or matrix inversion, but the actual model fitting and saving steps are commented out. GPU memory is managed and garbage collection is performed after processing each subject to ensure efficient resource usage.

In [ ]:
## Group-Level - Train ##
import numpy as np
import cupy as cp
import gc
import inout
from sklearn.linear_model import LinearRegression
from lin_gen_model import Subject
from transformations import symmetric_modification
from inout import get_schaefer100_sc, get_schaefer100_fc

def train_group(subject_ids):
    
    transformed_scs = []
    transformed_fcs = []

    for subject_id in subject_ids:
        sc, fc = get_schaefer100_sc(subject_id), get_schaefer100_fc(subject_id)
        sc = cp.log10(sc + 1)
        subject = Subject(subject_id, sc, fc, symmetric_modification)

        transformed_scs.append(subject.transformed_sc.copy())
        transformed_fcs.append(subject.transformed_fc.copy())

        del sc, fc, subject
        cp.get_default_memory_pool().free_all_blocks()
        cp.get_default_pinned_memory_pool().free_all_blocks()
        # Force Python garbage collection:
        gc.collect()

    X = cp.vstack(transformed_scs).T
    y = cp.hstack(transformed_fcs)

    del transformed_scs, transformed_fcs
    cp.get_default_memory_pool().free_all_blocks()
    cp.get_default_pinned_memory_pool().free_all_blocks()
    # Force Python garbage collection:
    gc.collect()

    a = cp.linalg.pinv(transformed_scs[:6555, :6555*30])

    #rules = LinearRegression(fit_intercept=False).fit(transformed_scs.T.astype(np.float32), transformed_fcs.astype(np.float32))

    # Save the model
    #sl_dir = "group_level_log10"
    #inout.check_paths(sl_dir)
    #np.savetxt(f"{sl_dir}/rules", rules)

subject_ids = [subject_id for subject_id in range(1, 51)]
train_group(subject_ids)

This cell implements subject-level model training for both schizophrenia (schz) and control (cntrl) groups using connectome data stored in an HDF5 file. For each subject in both groups, it loads the subject's structural connectivity (SC) and functional connectivity (FC) matrices, applies a symmetric transformation, and fits a model using a binary search to optimize the regularization parameter. The resulting transformation rules are saved for each subject in group-specific directories. This workflow enables comparison of learned connectivity transformations between schizophrenia and control subjects.

In [ ]:
## Schizophrenia Subject-Level - Train ##
import h5py
import cupy as cp
from lin_gen_model import Subject, binary_search_train
from transformations import symmetric_modification
import inout


sc_ctrl_path = "SC_FC_Connectomes/SC_number_of_fibers/ctrl"
sc_schz_path = "SC_FC_Connectomes/SC_number_of_fibers/schz"
fc_ctrl_path = "SC_FC_Connectomes/FC_correlation/ctrl"
fc_schz_path = "SC_FC_Connectomes/FC_correlation/schz"
sc_dens_ctrl_path = "SC_FC_Connectomes/SC_density/ctrl"
sc_dens_schz_path = "SC_FC_Connectomes/SC_density/schz"

def get_mat(path, subject_id):
    # Open the HDF5 file
    with h5py.File("27_SCHZ_CTRL_dataset.mat", "r") as f:
        # Access the object reference datasets
        refs = f[path]

        # Dereference the objects
        data = [cp.array(f[ref]) for ref in refs[:,0]][1][subject_id - 1]
    return data

for group in ["schz", "cntrl"]:
    for subject_id in range(1,28):
        # Load the data for the subject
            if group == "schz":            
                sc = get_mat(sc_schz_path, subject_id)
                fc = get_mat(fc_schz_path, subject_id)
            elif group == "cntrl":
                sc = get_mat(sc_ctrl_path, subject_id)
                fc = get_mat(fc_ctrl_path, subject_id)
            cp.fill_diagonal(fc, 1)

            # Create the subject object.
            subject = Subject(subject_id, sc, fc, symmetric_modification)

            # Make sure the output directory exists. If not created it.
            outdir = f"schz_results/{group}/"
            inout.check_paths(outdir)

            # Train
            rules, alpha = binary_search_train(subject.transformed_sc, subject.transformed_fc)
            
            print(f"Subject {subject_id} finished training with alpha: {alpha}")
            print()

            cp.savetxt(outdir + f"rules_sub-{subject_id}", rules)  